In [3]:
# !pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [8]:
from langchain_openai import ChatOpenAI

# Tavily - a search engine
from langchain_community.tools.tavily_search import TavilySearchResults

from dotenv import load_dotenv

load_dotenv()

True

In [7]:
search = TavilySearchResults(max_results=2) 

search_results = search.invoke("what is the weather in SF")
print(search_results)

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1731691457, 'localtime': '2024-11-15 09:24'}, 'current': {'last_updated_epoch': 1731690900, 'last_updated': '2024-11-15 09:15', 'temp_c': 12.2, 'temp_f': 54.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 59, 'cloud': 25, 'feelslike_c': 10.9, 'feelslike_f': 51.5, 'windchill_c': 7.4, 'windchill_f': 45.3, 'heatindex_c': 9.6, 'heatindex_f': 49.3, 'dewpoint_c': 6.6, 'dewpoint_f': 43.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.6, 'gust_mph': 11.4, 'gust_kph': 18.3}}"}, {'url': 'https://www.timeanddate.com/weather/

In [9]:
llm = ChatOpenAI(model='gpt-3.5-turbo')

# TavilySearchResults can be used as a tool
tools = [search]
llm_with_tools = llm.bind_tools(tools)

In [10]:
result = llm_with_tools.invoke("what is the weather in SF?")
result.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in San Francisco'},
  'id': 'call_i1li9E2lvpimrJZv7mM5kX9n',
  'type': 'tool_call'}]

In [15]:
# note that the llm did not returned the expected output. For that, one ways is to use agents
result.content

''

### Create an agent

we use `langgraph` to create an agent.

In [11]:
from langgraph.prebuilt import create_react_agent

In [13]:
agent_executer = create_react_agent(
    llm,
    tools
)
agent_executer

### Run an agent

-note that here the agent is stateless (or no memory)

In [19]:
from langchain_core.messages import HumanMessage

response = agent_executer.invoke({'messages': [HumanMessage(content='Where was Albert Einstein born?')]})
response['messages'][-1].content

'Albert Einstein was born on March 14, 1879, in Ulm, Württemberg, Germany.'

__We can check out the LangSmith trace to make sure it's calling the search tool effectively.__

https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r

## Adding memory

In order to the agent to remember previous interactions, we need to add memory. To give it memory we need to pass in a checkpointer. 

In [20]:
from langgraph.checkpoint.memory import MemorySaver

In [21]:
memory = MemorySaver()

__For each conversion we should define a different `thread_id`__ 

In [33]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)
config = {'configurable': {"thread_id": 'one'}}

response = agent_executor.invoke(
    {
        'messages':[HumanMessage(content="Hi! My name is Mohammad!")]
    }, 
    config
)
response['messages'][-1].content

'Hello Mohammad! How can I assist you today?'

In [34]:
response2 = agent_executor.invoke(
    {
        'messages': [HumanMessage(content='What is my name?')]
    },
    config
)
response2['messages'][-1].content

'Your name is Mohammad. How can I help you, Mohammad?'

#### Streaming messages

If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [42]:
for chunk in agent_executer.stream(
    {
        'messages':[HumanMessage(content="whats the weather in sf?")]
    }, 
    config
):
    print(chunk)
    print("...............")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0kWbvXDwlCIfADY3NL8rqc7M', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-24ed2587-f68a-4b76-a0cb-f0957ff662d7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_0kWbvXDwlCIfADY3NL8rqc7M', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109, 'input_token_details': {'a